## Exploration of explicitly encoding Lexical Patterns into Neural Models

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
%ls

In [ ]:
%ls ..

In [2]:
import numpy as np
import h5py
import composition_learning
from gensim.models import KeyedVectors as embedding_model

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1050 Ti (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5110)


Zunächst erstmal Zielnomen, Nomen, Adjektive aus tsv-File laden

In [3]:
import csv
path_to_adj_noun_file = "../data/amod.train.filtered"
gnoun_noun_adj_list = [] #a list of lists, each sublist containing a goal-noun, noun, adjective triple
goal_noun_list = [] #only the goal nouns, needed for functions from composition_learning.py.
with open(path_to_adj_noun_file, "r") as tsv_file:
    tsv_reader = csv.reader(tsv_file, delimiter='\t')
    for row in tsv_reader:
        gnoun_noun_adj_list.append(row)
        goal_noun_list.append(row[0].upper())

Danach Word Embeddings aus Modell laden:

In [20]:
path_to_word_embeddings = "../data/GoogleNews-vectors-negative300.bin"
vector_space = embedding_model.load_word2vec_format(path_to_word_embeddings, binary=True)

Daten in das richtige Format für das Keras-Modell bringen:

In [ ]:
training_data, training_labels, not_in_list = composition_learning.construct_data_and_labels(gnoun_noun_adj_list,
                                                                                             vector_space,
                                                                                             goal_noun_list,
                                                                                             verbosity = 0)


In [6]:
print("Shape of training data: {}".format(np.shape(training_data)))
print("Shape of training labels: {}".format(np.shape(training_labels)))

Shape of training data: (16198, 2, 300)
Shape of training labels: (16198, 300)


In [5]:
f.close()

In [ ]:
f = h5py.File('../data/lexical_training_data.hdf5', 'w')
f.create_dataset('training_data', data=training_data)
f.create_dataset('training_labels', data=training_labels)
f.close()

In [20]:
??g.close

In [6]:
g = h5py.File('../data/lexical_training_data.hdf5', 'r')
training_data = g["training_data"]
training_labels = g["training_labels"]


In [7]:
print(np.shape(training_data))
print(np.shape(training_labels))

(16198, 2, 300)
(16198, 300)


In [8]:
def training_data_generator(training_data, training_labels, batch_size):
    num_batches = int(np.floor(np.shape(training_data)[0] / batch_size))
    i=0
    while i < num_batches*batch_size:
        gen_output = (training_data[i:i+batch_size], training_labels[i:i+batch_size])
        #gen_output = ([training_data[i:i+batch_size,0,:],training_data[i:i+batch_size,1,:]], training_labels[i:i+batch_size])
        #print(i, np.shape(gen_output))
        i += batch_size
        if i >= (num_batches*batch_size): i = 0
        #print(i)
        yield gen_output
#TODO: nimmt insgesamt zu wenige samples pro epoche


In [ ]:
gen = training_data_generator(t_data, t_labels, batch_size=64)
print(np.shape(next(gen)[1]))

Das Modell trainieren. tensor_model bezeichnet das modell, indem adjektiv mit Tensor zusammen zu einer Matrix multipliziert wird, die dann mit Nomen-vektor multipliziert wird um wieder einen Vektor zu erhalten.
weighted_model steht für ein additives Modell mit Gewichtigunsmatrizen für je Adjetiv und Nomen.

In [ ]:
print(np.shape(gnoun_noun_adj_list))
print(len(not_in_list))
print(np.shape(training_data), np.shape(training_labels))

In [9]:
batch_size = 256

In [10]:
tensor_model = composition_learning.train_model(training_data_generator(training_data, training_labels, batch_size=batch_size), 
                                                steps_per_epoch = np.floor(np.shape(training_data)[0] / batch_size),
                                                composition_mode = 'tensor_mult_identity', verbosity=2)

Trainiere NN mit mode tensor_mult_identity...


/home/fabian/work/ag_sc/repeval2017/repeval2017/composition_learning.py:229: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=63.0, verbose=2, epochs=10)`
  model.fit_generator(data_generator,steps_per_epoch=steps_per_epoch, verbose=verbosity, nb_epoch=nb_epoch)


Epoch 1/10
8s - loss: -3.6268e-04 - acc: 0.0110
Epoch 2/10
8s - loss: -9.6484e-04 - acc: 0.0209
Epoch 3/10
8s - loss: -1.0975e-03 - acc: 0.0263
Epoch 4/10
8s - loss: -1.1757e-03 - acc: 0.0305
Epoch 5/10
8s - loss: -1.2301e-03 - acc: 0.0339
Epoch 6/10
8s - loss: -1.2711e-03 - acc: 0.0367
Epoch 7/10
8s - loss: -1.3038e-03 - acc: 0.0393
Epoch 8/10
8s - loss: -1.3306e-03 - acc: 0.0418
Epoch 9/10
8s - loss: -1.3534e-03 - acc: 0.0442
Epoch 10/10
8s - loss: -1.3730e-03 - acc: 0.0462


In [24]:
tensor_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 2, 300)            0         
_________________________________________________________________
magic_operation_2 (MagicOper (None, 300)               27000000  
Total params: 27,000,000
Trainable params: 27,000,000
Non-trainable params: 0
_________________________________________________________________


In [11]:
weighted_model = composition_learning.train_model(training_data_generator(training_data, training_labels, batch_size=batch_size), 
                                                steps_per_epoch = int(np.floor(np.shape(training_data)[0] / batch_size)),
                                                composition_mode = 'weighted_adj_and_noun_add_identity', nb_epoch=100, verbosity=2)


Trainiere NN mit mode weighted_adj_and_noun_add_identity...


/home/fabian/work/ag_sc/repeval2017/repeval2017/composition_learning.py:229: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=63, verbose=2, epochs=100)`
  model.fit_generator(data_generator,steps_per_epoch=steps_per_epoch, verbose=verbosity, nb_epoch=nb_epoch)


Epoch 1/100
0s - loss: -1.2208e-03 - acc: 0.0355
Epoch 2/100
0s - loss: -1.3556e-03 - acc: 0.0435
Epoch 3/100
0s - loss: -1.4057e-03 - acc: 0.0497
Epoch 4/100
0s - loss: -1.4372e-03 - acc: 0.0538
Epoch 5/100
0s - loss: -1.4603e-03 - acc: 0.0567
Epoch 6/100
0s - loss: -1.4782e-03 - acc: 0.0577
Epoch 7/100
0s - loss: -1.4927e-03 - acc: 0.0584
Epoch 8/100
0s - loss: -1.5046e-03 - acc: 0.0601
Epoch 9/100
0s - loss: -1.5147e-03 - acc: 0.0616
Epoch 10/100
0s - loss: -1.5233e-03 - acc: 0.0621
Epoch 11/100
0s - loss: -1.5307e-03 - acc: 0.0626
Epoch 12/100
0s - loss: -1.5372e-03 - acc: 0.0634
Epoch 13/100
0s - loss: -1.5428e-03 - acc: 0.0641
Epoch 14/100
0s - loss: -1.5478e-03 - acc: 0.0644
Epoch 15/100
0s - loss: -1.5522e-03 - acc: 0.0650
Epoch 16/100
0s - loss: -1.5561e-03 - acc: 0.0654
Epoch 17/100
0s - loss: -1.5596e-03 - acc: 0.0654
Epoch 18/100
0s - loss: -1.5628e-03 - acc: 0.0655
Epoch 19/100
0s - loss: -1.5656e-03 - acc: 0.0660
Epoch 20/100
0s - loss: -1.5681e-03 - acc: 0.0663
Epoch 21/

In [ ]:
??composition_learning.train_model

In [14]:
weighted_model.name = "weighted_model_new"
tensor_model.name = "tensor_model_new"

Modelle abspeichern für später(funktioniert irgendwie noch nicht, wahrscheinlich wegen eigener layerdefinition):

In [ ]:
%mkdir ../models/

In [ ]:
weighted_model.save_weights("../models/weighted_model_weights_new", overwrite=True)
tensor_model.save_weights("../models/tensor_model_weights_new", overwrite=True)

In [ ]:
%cd ../repeval2017/

Modelle wieder laden:

In [16]:
init = composition_learning.init_3d_identity((10,10,10))
print(type(init))

(10, 10, 10)
<class 'theano.sandbox.cuda.var.CudaNdarraySharedVariable'>


In [ ]:
weighted_model = composition_learning.create_model(composition_mode = 'weighted_adj_and_noun_add_identity')
weighted_model.load_weights("../models/weighted_model_weights")
tensor_model = composition_learning.create_model(composition_mode = 'tensor_mult_identity')
#print(tensor_model.output_shape)

#tensor_model.summary()
tensor_model.load_weights("../models/tensor_model_weights")

weighted_model.name = "weighted_model"
tensor_model.name = "tensor_model"

Um mit den Modellen Vektoren zu berechnen, tut man folgendes:

In [ ]:
goal_noun, noun, adj = gnoun_noun_adj_list[0]


goal_noun_tensor_vector = tensor_model.predict(np.asarray([[vectorspace[adj], vectorspace[noun]]]))[0, 0]
goal_noun_weighted_add_vector = weighted_model.predict(np.asarray([[vectorspace[adj], vectorspace[noun]]]))[0]

### Evaluierung

Lade Datensatz von hier: http://jair.org/media/3640/live-3640-6413-jair.txt

In [ ]:
%cd ../data/
!wget "http://jair.org/media/3640/live-3640-6413-jair.txt"
%ls
%cd ../repeval2017/

Evaluierungsdatensatz einlesen:

In [17]:
import csv
path_to_eval_file = "../data/live-3640-6413-jair.txt"
data = []
with open(path_to_eval_file, "r") as tsv_file:
    tsv_reader = csv.reader(tsv_file, delimiter='|')
    i = 0    
    for row in tsv_reader:
        if row[0][0] != '#':
            data.append([])
            for elem in row:
                data[i].append(elem.strip())
            i+=1

training_data = data[0:680] #Split nach Angabe von Turney
test_data = data[680:]
print(len(training_data))
print(len(test_data))

680
1500


In [ ]:
??complete_vector_space

In [22]:
from scipy.spatial.distance import cosine
import sys

def compute_distances_to_candidates(model, vector_space, sample, complete_vector_space=False, topn=150):
    #print("Processing {}".format(sample))
    distances = []
    not_in_vector_space = []
    
    phrase = sample[0]
    adj,noun = phrase.split()
    # berechne phrasenvektor mit modell
    #print("Complete_vector_space = {}".format(complete_vector_space))
    try:      
        if 'weighted_model' in model.name: phrase_vec = model.predict(np.asarray([[vector_space[adj], vector_space[noun]]]))[0]
        elif 'tensor_model' in model.name: phrase_vec = model.predict(np.asarray([[vector_space[adj], vector_space[noun]]]))[0,0]
    except:
        not_in_vector_space.append([adj,noun])
        return distances
    
    if complete_vector_space:
        #print(np.shape(phrase_vec))
        distances = vector_space.most_similar(positive=[phrase_vec], topn=topn)
    elif not complete_vector_space:
        for i in range(1,len(sample)):
            # berechne vektoren für alle targets per look up
            try:
                target_vec = vector_space[sample[i]]
                distance = cosine(phrase_vec, target_vec)
                distances.append((sample[i], distance))
            except KeyError as e:
                distances.append((sample[i], np.infty))#wörter, die nicht im vektorraum enthalten sind, werden als unendlich weit weg behandelt
                not_in_vector_space.append(sample[i])
                #print("{} is not in vectorspace".format(sample[i]))
            
    #sortiere nach distanz
    sorted_distances = sorted(distances, key=lambda x:x[1], reverse=False)
    #print(sorted_distances)
    return sorted_distances
    
def eval_model(model, vector_space, test_data, mode='average_rank', k=1, topn=100):
    if mode == 'average_rank':
        ranks = []
        empty_list_counter = 0
        for sample in test_data:            
            distances = compute_distances_to_candidates(model, vector_space, sample)
            if distances:
                for i in range(0, len(distances)):
                    if distances[i][0] == sample[1]: #find rank of gold label
                        #print(distances)
                        ranks.append(i)
            else:
                empty_list_counter += 1
        print("Could not compute rank in {} of {} cases due to empty distance lists.\n(Probably due to words not appearing in the embedding space.)".format(empty_list_counter, len(test_data)))
        return np.mean(ranks)
    elif mode == 'average_rank_complete_vectorspace':
        ranks = []
        for sample in test_data:
            distances = compute_distances_to_candidates(model, vector_space, sample, complete_vector_space=True, topn=topn)
            if distances:
                for i in range(0, len(distances)):
                    if distances[i][0] == sample[1]: #find rank of gold label
                        #print(distances)
                        ranks.append(i)
                    else:
                        ranks.append(topn + 1)
            else:
                empty_list_counter += 1
        print("Could not compute rank in {} of {} cases due to empty distance lists.\n(Probably due to words not appearing in the embedding space.)".format(empty_list_counter, len(test_data)))
        return np.mean(ranks)
    elif mode == 'accuracy':
        correct_counter = 0.0
        empty_list_counter = 0
        for sample in test_data:
            distances = compute_distances_to_candidates(model, vector_space, sample)
            if distances: 
                if distances[0][0] == sample[1]: #gold label is rank 1
                    correct_counter += 1
                else:
                    correct_counter += 5/7
            else:
                empty_list_counter += 1
                
        print("Could not compute rank in {} of {} cases due to empty distance lists.(Probably due to words not appearing in the embedding space.)".format(empty_list_counter, len(test_data)))
        return (correct_counter / (len(test_data) - empty_list_counter))
    elif mode == 'precision':
        correct_counter = 0
        empty_list_counter = 0
        for sample in test_data:
            distances = compute_distances_to_candidates(model, vector_space, sample)
            if distances:
                k_nearest = [target for target, distance in distances]
                if sample[1] in k_nearest[:k]:
                    correct_counter += 1
            else:
                empty_list_counter += 1
        return (correct_counter / (len(test_data) - empty_list_counter))
                


In [23]:
print("--------------------------") 
print("{}: Average rank of the gold label: {}".format(weighted_model.name.upper(), eval_model(weighted_model, vector_space, test_data, mode='average_rank')))
print("--------------------------")
print("{}: Accuracy of the model: {}".format(weighted_model.name.upper(), eval_model(weighted_model, vector_space, test_data, mode='accuracy')))
print("--------------------------")
print("{}: Precision_at_1 of the model: {}".format(weighted_model.name.upper(), eval_model(weighted_model, vector_space, test_data, mode='precision')))
print("--------------------------") 
print("{}: Precision_at_2 of the model: {}".format(weighted_model.name.upper(), eval_model(weighted_model, vector_space, test_data, mode='precision', k=2)))
print("--------------------------") 
print("{}: Precision_at_3 of the model: {}".format(weighted_model.name.upper(), eval_model(weighted_model, vector_space, test_data, mode='precision', k=3)))
print("--------------------------") 
print("{}: Average rank of the gold label: {}".format(tensor_model.name.upper(), eval_model(tensor_model, vector_space, test_data, mode='average_rank')))
print("--------------------------")
print("{}: Accuracy of the model: {}".format(tensor_model.name.upper(), eval_model(tensor_model, vector_space, test_data, mode='accuracy')))
print("--------------------------")
print("{}: Precision_at_1 of the model: {}".format(tensor_model.name.upper(), eval_model(weighted_model, vector_space, test_data, mode='precision')))
print("--------------------------")
print("{}: Precision_at_2 of the model: {}".format(tensor_model.name.upper(), eval_model(weighted_model, vector_space, test_data, mode='precision', k=2)))
print("--------------------------")
print("{}: Precision_at_3 of the model: {}".format(tensor_model.name.upper(), eval_model(weighted_model, vector_space, test_data, mode='precision', k=3)))


--------------------------
Could not compute rank in 18 of 1500 cases due to empty distance lists.
(Probably due to words not appearing in the embedding space.)
WEIGHTED_MODEL_NEW: Average rank of the gold label: 2.6214574898785425
--------------------------
Could not compute rank in 18 of 1500 cases due to empty distance lists.(Probably due to words not appearing in the embedding space.)
WEIGHTED_MODEL_NEW: Accuracy of the model: 0.7337574705995608
--------------------------
WEIGHTED_MODEL_NEW: Precision_at_1 of the model: 0.06815114709851552
--------------------------
WEIGHTED_MODEL_NEW: Precision_at_2 of the model: 0.3083670715249663
--------------------------
WEIGHTED_MODEL_NEW: Precision_at_3 of the model: 0.5654520917678812
--------------------------
Could not compute rank in 18 of 1500 cases due to empty distance lists.
(Probably due to words not appearing in the embedding space.)
TENSOR_MODEL_NEW: Average rank of the gold label: 2.591093117408907
--------------------------
Coul

In [ ]:
print("{}: Average rank of the gold label: {} (whole vectorspace!)".format(weighted_model.name.upper(), eval_model(weighted_model, vector_space, test_data, mode='average_rank_complete_vectorspace')))

In [20]:
i = 1 
i += 5/7
print(i)

1.7142857142857144
